
# RE2020 – Générateur de graphiques (Jupyter lab + Voila + Binder)
## Outil en développement, version du 24/02/2026

**Nouveautés v11**
- ✅ Axe X **identique** pour **CEP** et **CEPnr**
- ✅ Option **CEPnr = CEP** (copie toutes les valeurs CEP vers CEPnr, **sauf le max**) + verrouillage des champs CEPnr
- ✅ Notebook **multi-cellules** (imports / widgets / calculs / plots / UI séparés)
- ✅ Ajout de Ic,construction
- ✅ DH au dessus de IcE
- ✅ Donner la main aux utilisateurs sur l'alpha du fond

**To Do**
- Ajuster les tuiles sur le Récap, pas assez de marges actuellement.
- Réduire les marges sur les PNG, elles sont énorme (afficher une couleur de fond pour aide)
- Ajouter contour sur le fond ? pour rendre les catégories plus lisibles
- Ajuster couleur la plus claire des CEP (trop clair)
- Charte WSP ? Pas terrible les couleurs je trouve, à voir
- Ajouter un texte CONFORME RE2020 en gras et vert à la fin du titre ? NON on annule.
- Ajout d'un Dashboard récap résultats

In [248]:

import ipywidgets as widgets
from IPython.display import display, FileLink, Image
from PIL import Image as PILImage, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.patches import FancyBboxPatch
import math

In [249]:
# === Widgets pour paramètres supplémentaires ===
nbRange_widget = widgets.IntText(description="Nb Range :", value=5)
HeightBar_widget = widgets.IntText(description="Hauteur Bar :", value=6)
heightFond_widget = widgets.IntText(description="Hauteur Fond :", value=18)
graduationx_widget = widgets.Text(
    description="Graduation axe x (bbio, cep, dh, cepnr, ic énergie, ic construction) :",
    value="5,100,10,10,10,50",
    placeholder="Ex: 5,100,10,10,10,50"
)
AlphaFond_widget = widgets.FloatText(description="Transparence fond :",placeholder="entre 0 et 1", value=0.4)

titre_widget = widgets.Textarea(description="Titre :", value="RE2020 - BlaBlaBla le Titre du Projet")
sousTitre_widget = widgets.Textarea(description="Sous-titre :", value="[PHASE] BlaBlaBla l'objectifs machin est atteint")
infos_widget = widgets.Textarea(description="Infos :", value="Projet Machin Machin - Phase XXX - Calcul du XX/XX/20XX - Pléiades version 6.25.X.X - moteur RE 2024.E1.0.0 - WSP France ©")
note = (
    "Aide à la lecture : chaque barre centrale correspond à la valeur totale de l’indicateur, obtenue par la somme de ses composantes.\n"
    "Les aplats en arrière‑plan découpent le seuil réglementaire maximal en parts égales, ce qui permet de visualiser rapidement "
    "le niveau de performance et le pourcentage de réduction par rapport au maximum autorisé.\n"
    "La position de la barre par rapport au trait de seuil indique l’atteinte (ou non) des objectifs réglementaires."
)



In [250]:
# === Widgets pour indicateurs ===
BbioChaud_widget = widgets.FloatText(description="Bbio Chaud :", value=40.4)
BbioFroid_widget = widgets.FloatText(description="Bbio Froid :", value=24.8)
BbioEcl_widget = widgets.FloatText(description="Bbio Eclairage :", value=20.5)
Bbiomax_widget = widgets.FloatText(description="Seuil Bbio :", value=104.2)

CEPch_widget = widgets.FloatText(description="CEP Chauffage :", value=16.1)
CEPfr_widget = widgets.FloatText(description="CEP Climatisation :", value=9.7)
CEPecs_widget = widgets.FloatText(description="CEP ECS :", value=2.5)
CEPecl_widget = widgets.FloatText(description="CEP Éclairage :", value=5.5)
CEPauxV_widget = widgets.FloatText(description="CEP Auxiliaires de Ventilation :", value=17)
CEPauxD_widget = widgets.FloatText(description="CEP Auxiliaires de Distribution :", value=1.1)
CEPdep_widget = widgets.FloatText(description="CEP Déplacements :", value=4.4)
CEPmax_widget = widgets.FloatText(description="Seuil CEP :", value=86.7)

CEPnrch_widget = widgets.FloatText(description="CEPnr Chauffage :", value=16.1)
CEPnrfr_widget = widgets.FloatText(description="CEPnr Climatisation :", value=9.7)
CEPnrecs_widget = widgets.FloatText(description="CEPnr ECS :", value=2.5)
CEPnrecl_widget = widgets.FloatText(description="CEPnr Éclairage :", value=5.5)
CEPnrauxV_widget = widgets.FloatText(description="CEPnr Auxiliaires de Ventilation :", value=17)
CEPnrauxD_widget = widgets.FloatText(description="CEPnr Auxiliaires de Distribution :", value=1.1)
CEPnrdep_widget = widgets.FloatText(description="CEPnr Déplacements :", value=4.4)
CEPnrmax_widget = widgets.FloatText(description="Seuil CEPnr :", value=76.5)

# Option: rendre CEPnr = CEP (copie valeurs CEP vers CEPnr, sauf le max)
cepnr_equals_cep_widget = widgets.Checkbox(value=True, description="CEPnr = CEP")

IcEelec_widget = widgets.FloatText(description="IcE Électricité :", value=66.2)
IcEbois_widget = widgets.FloatText(description="IcE Bois :", value=0)
IcErcu_widget = widgets.FloatText(description="IcE Réseau chaleur/froid :", value=0)
IcEfioul_widget = widgets.FloatText(description="IcE Fioul :", value=0)
IcEgaz_widget = widgets.FloatText(description="IcE Gaz :", value=0)
IcEmax_widget = widgets.FloatText(description="Seuil IcE :", value=204)

# === Ic construction (saisie par lots) ===
IcC_lot01_widget = widgets.FloatText(description="lot 01 - voirie et réseaux divers :", value=12.0)
IcC_lot02_widget = widgets.FloatText(description="lot 02 - infrastructures :", value=25.0)
IcC_lot03_widget = widgets.FloatText(description="lot 03 - superstructure :", value=220.0)
IcC_lot04_widget = widgets.FloatText(description="lot 04 - couverture :", value=40.0)
IcC_lot05_widget = widgets.FloatText(description="lot 05 - cloisonnement :", value=18.0)
IcC_lot06_widget = widgets.FloatText(description="lot 06 - façade :", value=95.0)
IcC_lot07_widget = widgets.FloatText(description="lot 07 - revêtements :", value=25.0)
IcC_lot08_widget = widgets.FloatText(description="lot 08 - chauffage ventilation climatisation :", value=70.0)
IcC_lot09_widget = widgets.FloatText(description="lot 09 - plomberie :", value=18.0)
IcC_lot10_widget = widgets.FloatText(description="lot 10 - courants forts :", value=20.0)
IcC_lot11_widget = widgets.FloatText(description="lot 11 - courants faibles :", value=10.0)
IcC_lot12_widget = widgets.FloatText(description="lot 12 - ascenseurs :", value=6.0)
IcC_lot13_widget = widgets.FloatText(description="lot 13 - photovoltaïque :", value=0.0)
IcCmax_widget = widgets.FloatText(description="Seuil Ic construction :", value=710.0)

show_icc_widget = widgets.Checkbox(value=True, description="Afficher Ic construction")
DH_widget = widgets.FloatText(description="DH :", value=1104.8)
DHmax_widget = widgets.FloatText(description="Seuil DH :", value=1150)

show_bbio_widget = widgets.Checkbox(value=True, description="Afficher Bbio")
show_cep_widget = widgets.Checkbox(value=True, description="Afficher CEP")
show_cepnr_widget = widgets.Checkbox(value=True, description="Afficher CEPnr")
show_ice_widget = widgets.Checkbox(value=True, description="Afficher IcE")
show_dh_widget = widgets.Checkbox(value=True, description="Afficher DH")


In [251]:
# === Sync CEP -> CEPnr si option activée ===
cep_to_cepnr_map = {
    CEPch_widget: CEPnrch_widget,
    CEPfr_widget: CEPnrfr_widget,
    CEPecs_widget: CEPnrecs_widget,
    CEPecl_widget: CEPnrecl_widget,
    CEPauxV_widget: CEPnrauxV_widget,
    CEPauxD_widget: CEPnrauxD_widget,
    CEPdep_widget: CEPnrdep_widget,
}

def _apply_cep_to_cepnr():
    for w_src, w_dst in cep_to_cepnr_map.items():
        w_dst.value = w_src.value

def _toggle_cepnr_sync(*args):
    enabled = cepnr_equals_cep_widget.value
    for w_dst in cep_to_cepnr_map.values():
        w_dst.disabled = enabled
    if enabled:
        _apply_cep_to_cepnr()

cepnr_equals_cep_widget.observe(lambda ch: _toggle_cepnr_sync(), names='value')

def _on_cep_change(change):
    if cepnr_equals_cep_widget.value:
        for w_src, w_dst in cep_to_cepnr_map.items():
            if change['owner'] is w_src:
                w_dst.value = change['new']
                break

for w_src in cep_to_cepnr_map.keys():
    w_src.observe(_on_cep_change, names='value')

_toggle_cepnr_sync()


In [252]:
# === Helpers calculs ===

def _filter_nonzero(values, labels):
    pairs = [(v, l) for v, l in zip(values, labels) if v != 0]
    if pairs:
        v2, l2 = map(list, zip(*pairs))
        return v2, l2
    return [], []

# === Couleurs (vibes modernes) ===

# Teintes de base (0–360) : change juste ces valeurs si tu veux d’autres ambiances
BASE_HUE = {
    "Bbio": 160,   # teal/vert moderne
    "CEP": 250,    # indigo/violet
    "CEPnr": 270,  # IDENTIQUE à CEP
    "IcE": 35,     # orange/rouge doux
    "DH": 210      # bleu
}

# Fond coloré doux (tinted) par indicateur
# (on utilise des teintes Tailwind très claires + un point un peu plus soutenu) [1](https://wsponline.sharepoint.com/sites/CA-Buildings-BIM/SitePages/Page_ModelAutomation_AddIns_QuickReference_EN.aspx?web=1)[2](https://wsponline.sharepoint.com/sites/GLOBAL-ACC-Guidance/SitePages/Automation-Job-Processor-%28AJP%29-Workflows.aspx?web=1)

BG = {
    "Bbio":  ("#ccfbf1", "#5eead4"),  # teal-100 -> teal-300
    "CEP":   ("#e0e7ff", "#a5b4fc"),  # indigo-100 -> indigo-300
    "CEPnr": ("#e0e7ff", "#a5b4fc"),  # identique CEP
    "IcE":   ("#ffedd5", "#fdba74"),  # orange-100 -> orange-300
    "DH":    ("#e0f2fe", "#7dd3fc"),  # sky-100 -> sky-300
    "IcC":   ("#ffe4e6", "#fda4af"),  # rose-100 -> rose-300
}

def fond_couleur(label, nbRange, reverse=True):
    """
    Dégradé coloré doux (tinted).
    reverse=True => plus foncé à gauche, plus clair à droite (souvent plus joli sur une progression).
    """
    c1, c2 = BG.get(label, ("#f3f4f6", "#d1d5db"))  # fallback gris clair
    cols = sns.blend_palette([c1, c2], n_colors=nbRange)
    return cols[::-1] if reverse else cols


def palette_composants(n, base_h):
    """
    Palette moderne par composant (adaptative au nombre de composantes).
    husl -> couleurs tendances, bien espacées, cohérentes.
    """
    if n <= 0:
        return []
    return sns.husl_palette(n, h=base_h, s=0.75, l=0.55)


def legend_rowwise(handles, labels, ncol):
    """
    Réordonne handles/labels pour que la lecture visuelle se fasse par lignes
    (gauche -> droite), même si matplotlib remplit en colonnes.
    """
    m = len(labels)
    if ncol <= 1 or m <= 1:
        return handles, labels

    nrows = int(np.ceil(m / ncol))
    # On construit une liste "input" telle que l'affichage final soit row-major.
    new_h = [None] * m
    new_l = [None] * m

    # desired order = labels dans l'ordre naturel (row-major)
    k = 0
    for r in range(nrows):
        for c in range(ncol):
            q = r * ncol + c
            if q >= m:
                continue
            p = r + nrows * c  # position dans l'ordre colonne-major attendu
            if p < m:
                new_h[p] = handles[q]
                new_l[p] = labels[q]
                k += 1

    # fallback sécurité (si trous à cause de p>=m)
    new_h2, new_l2 = [], []
    for h, l in zip(new_h, new_l):
        if h is not None:
            new_h2.append(h); new_l2.append(l)

    return new_h2, new_l2


In [253]:
def generer_recap_re2020(d_bbio, d_CEP, d_DH, d_CEPnr, d_IcE, d_IcC,
                         titre, selected_keys=None, dpi=250):

    # ====== Taille image ======
    FIG_W_CM = 22
    FIG_H_CM = 8.5
    W = int(FIG_W_CM / 2.54 * dpi)
    H = int(FIG_H_CM / 2.54 * dpi)

    # Fond global (blanc demandé)
    BG = (255, 255, 255)

    # ====== Couleurs ======
    GREEN = (22, 163, 74)
    RED   = (220, 38, 38)
    DARK_GREY = (55, 65, 81)

    # Fond des tuiles selon conformité
    GOOD_TILE = (236, 253, 245)  # vert très clair
    BAD_TILE  = (254, 242, 242)  # rouge très clair

    # Bordure tuile
    TILE_EDGE = (160, 174, 192)
    TILE_EDGE_W = max(2, int(2 * dpi / 150))
    RADIUS = int(18 * dpi / 250)

    # ====== Layout ======
    # marges
    M_L = int(22 * dpi / 250)
    M_R = int(16 * dpi / 250)
    M_T = int(18 * dpi / 250)
    M_B = int(16 * dpi / 250)

    # gaps entre tuiles
    GAP_X = int(16 * dpi / 250)
    GAP_Y = int(14 * dpi / 250)

    # espacement titre <-> grille (évite que le titre passe sous les tuiles)
    TITLE_GAP = int(45 * dpi / 250)

    # ====== Typo (tailles) ======
    FS_TITLE = int(100 * dpi / 250)   # gros
    FS_LABEL = int(60 * dpi / 250)   # plus gros
    FS_PCT   = int(100 * dpi / 250)   # beaucoup plus gros
    FS_VAL   = int(40 * dpi / 250)   # plus gros

    # distance entre lignes dans une tuile (resserré)
    LINE_GAP = int(10 * dpi / 250)

    # ====== Polices (bold = police bold) ======
    def load_font(size, weight="regular"):
        try:
            import matplotlib.font_manager as fm
            prop = fm.FontProperties(family="DejaVu Sans",
                                     weight=("bold" if weight == "bold" else "normal"))
            path = fm.findfont(prop, fallback_to_default=True)
            return ImageFont.truetype(path, size=size)
        except Exception:
            return ImageFont.load_default()

    font_title_bold = load_font(FS_TITLE, "bold")
    font_label_bold = load_font(FS_LABEL, "bold")
    font_pct_bold   = load_font(FS_PCT,   "bold")
    font_val        = load_font(FS_VAL,   "regular")  # mets "bold" si tu veux encore plus

    # ====== Données ======
    def pct_vs_max(val, maxv):
        if maxv == 0:
            return 0.0
        return 100.0 * (val - maxv) / maxv

    items_all = [
        {"key": "Bbio",  "label": "Bbio",            "val": d_bbio["bbio"],   "max": d_bbio["Bbiomax"],   "unit": "pts"},
        {"key": "DH",    "label": "DH",              "val": d_DH["DH"],       "max": d_DH["DHmax"],       "unit": "°C·h"},
        {"key": "CEP",   "label": "CEP",             "val": d_CEP["CEP"],     "max": d_CEP["CEPmax"],     "unit": "kWhep/m².an"},
        {"key": "CEPnr", "label": "CEPnr",           "val": d_CEPnr["CEPnr"], "max": d_CEPnr["CEPnrmax"], "unit": "kWhep/m².an"},
        {"key": "IcE",   "label": "Ic énergie",      "val": d_IcE["IcE"],     "max": d_IcE["IcEmax"],     "unit": "kgCO₂e/m²"},
        {"key": "IcC",   "label": "Ic construction", "val": d_IcC["IcC"],     "max": d_IcC["IcCmax"],     "unit": "kgCO₂e/m²"},
    ]

    selected = items_all if selected_keys is None else [it for it in items_all if it["key"] in selected_keys]
    if not selected:
        return None

    conforme = all(it["val"] <= it["max"] for it in selected)
    status_txt = "RE2020 Conforme" if conforme else "RE2020 Non conforme"

    # ====== Canvas ======
    im = PILImage.new("RGB", (W, H), BG)
    dr = ImageDraw.Draw(im)

    # Mesure hauteur titre (fix du problème : titre sous tuiles)
    tb = dr.textbbox((0, 0), status_txt, font=font_title_bold)
    title_h = tb[3] - tb[1]

    # Zone grille (commence après le titre + gap)
    grid_left = M_L
    grid_right = W - M_R
    grid_top = M_T + title_h + TITLE_GAP
    grid_bottom = H - M_B

    # ====== Géométrie grille ======
    ncol = 3
    m = len(selected)
    nrow = math.ceil(m / ncol)

    avail_w = grid_right - grid_left
    avail_h = grid_bottom - grid_top

    tile_w = int((avail_w - (ncol - 1) * GAP_X) / ncol)
    tile_h = int((avail_h - (nrow - 1) * GAP_Y) / nrow)

    # helper: texte centré
    def draw_centered(text, cx, cy, font, fill):
        bbox = dr.textbbox((0, 0), text, font=font)
        tw = bbox[2] - bbox[0]
        th = bbox[3] - bbox[1]
        dr.text((cx - tw / 2, cy - th / 2), text, font=font, fill=fill)

    # ====== Dessin tuiles + textes (bloc resserré) ======
    for i, it in enumerate(selected):
        r = i // ncol
        c = i % ncol

        x0 = grid_left + c * (tile_w + GAP_X)
        y0 = grid_top + r * (tile_h + GAP_Y)
        x1 = x0 + tile_w
        y1 = y0 + tile_h

        ok = it["val"] <= it["max"]
        fill = GOOD_TILE if ok else BAD_TILE

        # tuile
        dr.rounded_rectangle(
            [x0, y0, x1, y1],
            radius=RADIUS,
            fill=fill,
            outline=TILE_EDGE,
            width=TILE_EDGE_W
        )

        # Textes
        cx = x0 + tile_w / 2
        p = pct_vs_max(it["val"], it["max"])
        p_txt = f"{p:+.0f}%"

        # % : vert si conforme indicateur, rouge sinon (plus logique visuellement)
        p_col = GREEN if ok else RED

        val_txt = f"{it['val']:.0f}/{it['max']:.0f} {it['unit']}"

        # Mesure hauteurs pour resserrer comme un bloc centré
        h_label = dr.textbbox((0, 0), it["label"], font=font_label_bold)[3]
        h_pct   = dr.textbbox((0, 0), p_txt,       font=font_pct_bold)[3]
        h_val   = dr.textbbox((0, 0), val_txt,     font=font_val)[3]

        total_h = h_label + LINE_GAP + h_pct + LINE_GAP + h_val
        start_y = y0 + (tile_h - total_h) / 2

        # positions des 3 lignes
        y_label = start_y + h_label / 2
        y_pct   = y_label + h_label / 2 + LINE_GAP + h_pct / 2
        y_val   = y_pct   + h_pct   / 2 + LINE_GAP + h_val / 2

        draw_centered(it["label"], cx, y_label, font_label_bold, DARK_GREY)
        draw_centered(p_txt,       cx, y_pct,   font_pct_bold,   p_col)
        draw_centered(val_txt,     cx, y_val,   font_val,       DARK_GREY)

    # ====== Dessin titre en dernier (garanti au-dessus) ======
    dr.text((M_L, M_T), status_txt, fill=GREEN, font=font_title_bold)

    out = f"{titre}_Recap_RE2020.png"
    im.save(out, "PNG")
    return out

In [254]:
# === Palettes dashboard (Tailwind-ish) ===
# Ordre: du plus foncé au plus clair (pour des segments lisibles)

TW = {
    "teal":  ["#115e59", "#0f766e", "#0d9488", "#14b8a6", "#2dd4bf", "#99f6e4"],   # 800..200
    "indigo":["#3730a3", "#4338ca", "#4f46e5", "#6366f1", "#818cf8", "#c7d2fe"],   # 800..200
    "orange":["#9a3412", "#c2410c", "#ea580c", "#f97316", "#fb923c", "#fed7aa"],   # 800..200
    "sky":   ["#075985", "#0369a1", "#0284c7", "#0ea5e9", "#38bdf8", "#bae6fd"],   # 800..200
    "rose":  ["#9f1239", "#e11d48", "#f43f5e", "#fb7185", "#fda4af", "#ffe4e6"],   # 800..200
}

# mapping indicateurs -> famille de couleurs
PALETTE_FAMILY = {
    "Bbio": "teal",
    "CEP": "indigo",
    "CEPnr": "indigo",  # identique CEP
    "IcE": "orange",
    "DH": "sky",
    "IcC": "rose",
}

def palette_composants(label, n):
    """Retourne n couleurs 'dashboard' pour un indicateur donné."""
    fam = PALETTE_FAMILY.get(label, "indigo")
    base = TW[fam]
    if n <= 0:
        return []
    # si n > 6, on étend doucement en interpolant avec seaborn
    if n <= len(base):
        return base[:n]
    return sns.blend_palette(base, n_colors=n)

# Optionnel (rend les segments plus lisibles)
SEGMENT_EDGE_COLOR = "white"
SEGMENT_EDGE_WIDTH = 0.4
#FOND_ALPHA = 0.4

In [255]:
# === Calcul des données ===

def calcul_donnees_bbio(nbRange):
    BbioChaud = BbioChaud_widget.value
    BbioFroid = BbioFroid_widget.value
    BbioEcl = BbioEcl_widget.value
    Bbiomax = Bbiomax_widget.value

    bbio = BbioChaud + BbioFroid + BbioEcl

    values = [BbioChaud, BbioFroid, BbioEcl]
    labels = [f"Chauffage : {BbioChaud}", f"Climatisation: {BbioFroid}", f"Eclairage: {BbioEcl}"]
    values, labels = _filter_nonzero(values, labels)

    colors = palette_composants("Bbio", len(values))
    extended_colors = fond_couleur("Bbio", nbRange, reverse=True)


    valuesFond = [Bbiomax / nbRange] * nbRange
    gaucheFond = np.cumsum(valuesFond) - valuesFond[0]

    return {
        "bbio": bbio,
        "valuesBbio": values,
        "labelsBbio": labels,
        "colorsBbio": colors,
        "extended_colorsBbio": extended_colors,
        "valuesFondBbio": valuesFond,
        "gaucheFondBbio": gaucheFond,
        "Bbiomax": Bbiomax
    }


def calcul_donnees_CEP(nbRange):
    CEPch = CEPch_widget.value
    CEPfr = CEPfr_widget.value
    CEPecs = CEPecs_widget.value
    CEPecl = CEPecl_widget.value
    CEPauxV = CEPauxV_widget.value
    CEPauxD = CEPauxD_widget.value
    CEPdep = CEPdep_widget.value
    CEPmax = CEPmax_widget.value

    CEP = CEPch + CEPfr + CEPecs + CEPecl + CEPauxV + CEPauxD + CEPdep

    values = [CEPch, CEPfr, CEPecs, CEPecl, CEPauxV, CEPauxD, CEPdep]
    labels = [
        f"Chauffage:{CEPch}", f"Clim.:{CEPfr}", f"ECS:{CEPecs}",
        f"Éclairage:{CEPecl}", f"Aux. ventil.:{CEPauxV}",
        f"Aux. distrib.:{CEPauxD}", f"Déplacements:{CEPdep}"
    ]
    values, labels = _filter_nonzero(values, labels)

    colors = palette_composants("CEP", len(values))
    extended_colors = fond_couleur("CEP", nbRange, reverse=True)
    
    valuesFond = [CEPmax / nbRange] * nbRange
    gaucheFond = np.cumsum(valuesFond) - valuesFond[0]

    return {
        "CEP": CEP,
        "valuesCEP": values,
        "labelsCEP": labels,
        "colorsCEP": colors,
        "extended_colorsCEP": extended_colors,
        "valuesFondCEP": valuesFond,
        "gaucheFondCEP": gaucheFond,
        "CEPmax": CEPmax
    }


def calcul_donnees_CEPnr(nbRange):
    # Si l'option est activée : CEPnr = CEP (sauf max)
    if cepnr_equals_cep_widget.value:
        CEPnrch = CEPch_widget.value
        CEPnrfr = CEPfr_widget.value
        CEPnrecs = CEPecs_widget.value
        CEPnrecl = CEPecl_widget.value
        CEPnrauxV = CEPauxV_widget.value
        CEPnrauxD = CEPauxD_widget.value
        CEPnrdep = CEPdep_widget.value
    else:
        CEPnrch = CEPnrch_widget.value
        CEPnrfr = CEPnrfr_widget.value
        CEPnrecs = CEPnrecs_widget.value
        CEPnrecl = CEPnrecl_widget.value
        CEPnrauxV = CEPnrauxV_widget.value
        CEPnrauxD = CEPnrauxD_widget.value
        CEPnrdep = CEPnrdep_widget.value

    CEPnrmax = CEPnrmax_widget.value

    CEPnr = CEPnrch + CEPnrfr + CEPnrecs + CEPnrecl + CEPnrauxV + CEPnrauxD + CEPnrdep

    values = [CEPnrch, CEPnrfr, CEPnrecs, CEPnrecl, CEPnrauxV, CEPnrauxD, CEPnrdep]
    labels = [
        f"Chauffage:{CEPnrch}", f"Climatisation:{CEPnrfr}", f"ECS:{CEPnrecs}",
        f"Éclairage:{CEPnrecl}", f"Aux. ventil.:{CEPnrauxV}",
        f"Aux. distrib.:{CEPnrauxD}", f"Déplacements:{CEPnrdep}"
    ]
    values, labels = _filter_nonzero(values, labels)

    colors = palette_composants("CEPnr", len(values))  # même famille que CEP
    extended_colors = fond_couleur("CEPnr", nbRange, reverse=True)  # identique CEP
    
    valuesFond = [CEPnrmax / nbRange] * nbRange
    gaucheFond = np.cumsum(valuesFond) - valuesFond[0]

    return {
        "CEPnr": CEPnr,
        "valuesCEPnr": values,
        "labelsCEPnr": labels,
        "colorsCEPnr": colors,
        "extended_colorsCEPnr": extended_colors,
        "valuesFondCEPnr": valuesFond,
        "gaucheFondCEPnr": gaucheFond,
        "CEPnrmax": CEPnrmax
    }


def calcul_donnees_IcE(nbRange):
    IcEelec = IcEelec_widget.value
    IcEbois = IcEbois_widget.value
    IcErcu = IcErcu_widget.value
    IcEfioul = IcEfioul_widget.value
    IcEgaz = IcEgaz_widget.value
    IcEmax = IcEmax_widget.value

    IcE = IcEelec + IcEbois + IcErcu + IcEfioul + IcEgaz

    values = [IcEelec, IcEbois, IcErcu, IcEfioul, IcEgaz]
    labels = [f"Elec:{IcEelec}", f"Bois:{IcEbois}", f"RCU:{IcErcu}", f"Fioul:{IcEfioul}", f"Gaz:{IcEgaz}"]
    values, labels = _filter_nonzero(values, labels)
    
    colors = palette_composants("IcE", len(values))
    extended_colors = fond_couleur("IcE", nbRange, reverse=True)

    valuesFond = [IcEmax / nbRange] * nbRange
    gaucheFond = np.cumsum(valuesFond) - valuesFond[0]

    return {
        "IcE": IcE,
        "valuesIcE": values,
        "labelsIcE": labels,
        "colorsIcE": colors,
        "extended_colorsIcE": extended_colors,
        "valuesFondIcE": valuesFond,
        "gaucheFondIcE": gaucheFond,
        "IcEmax": IcEmax
    }




def calcul_donnees_IcC(nbRange):
    """Ic construction : somme des lots (saisie détaillée), affichée en barre unique."""
    lots = [
        IcC_lot01_widget.value,
        IcC_lot02_widget.value,
        IcC_lot03_widget.value,
        IcC_lot04_widget.value,
        IcC_lot05_widget.value,
        IcC_lot06_widget.value,
        IcC_lot07_widget.value,
        IcC_lot08_widget.value,
        IcC_lot09_widget.value,
        IcC_lot10_widget.value,
        IcC_lot11_widget.value,
        IcC_lot12_widget.value,
        IcC_lot13_widget.value,
    ]

    IcC = float(np.sum(lots))
    IcCmax = IcCmax_widget.value

    values = [IcC]
    labels = [f"Total : {IcC}"]

    colors = palette_composants("IcC", len(values))
    extended_colors = fond_couleur("IcC", nbRange, reverse=True)

    valuesFond = [IcCmax / nbRange] * nbRange
    gaucheFond = np.cumsum(valuesFond) - valuesFond[0]

    return {
        "IcC": IcC,
        "valuesIcC": values,
        "labelsIcC": labels,
        "colorsIcC": colors,
        "extended_colorsIcC": extended_colors,
        "valuesFondIcC": valuesFond,
        "gaucheFondIcC": gaucheFond,
        "IcCmax": IcCmax
    }
def calcul_donnees_DH(nbRange):
    DH = DH_widget.value
    DHmax = DHmax_widget.value

    values = [DH]
    labels = [f"Degré heure : {DH}"]
    
    colors = palette_composants("DH", len(values))
    extended_colors = fond_couleur("DH", nbRange, reverse=True)

    valuesFond = [DHmax / nbRange] * nbRange
    gaucheFond = np.cumsum(valuesFond) - valuesFond[0]

    return {
        "DH": DH,
        "valuesDH": values,
        "labelsDH": labels,
        "colorsDH": colors,
        "extended_colorsDH": extended_colors,
        "valuesFondDH": valuesFond,
        "gaucheFondDH": gaucheFond,
        "DHmax": DHmax
    }


In [256]:
# === Affichage des graphes (layout spacers + CEP/CEPnr axe X commun) ===

def afficher_graphique(d_bbio, d_CEP, d_CEPnr, d_IcE, d_IcC, d_DH,
                       nbRange, HeightBar, heightFond, graduationx,
                       titre, sousTitre, infos):

    selected = []

    if show_bbio_widget.value: selected.append(("Bbio", d_bbio, graduationx[0]))

    if show_dh_widget.value: selected.append(("DH", d_DH, graduationx[1]))

    if show_cep_widget.value: selected.append(("CEP", d_CEP, graduationx[2]))

    if show_cepnr_widget.value: selected.append(("CEPnr", d_CEPnr, graduationx[3]))

    if show_ice_widget.value: selected.append(("IcE", d_IcE, graduationx[4]))

    if show_icc_widget.value: selected.append(("IcC", d_IcC, graduationx[5]))


    n = len(selected)
    if n == 0:
        print("Aucun indicateur sélectionné")
        return

    # Axe X commun CEP/CEPnr
    max_raw_cep = max(d_CEP.get('CEPmax', 0), d_CEP.get('CEP', 0), d_CEPnr.get('CEPnrmax', 0), d_CEPnr.get('CEPnr', 0))
    grad_common = min(graduationx[1], graduationx[3])
    xlim_cep = 1.05 * max_raw_cep
    xticks_cep = np.arange(0, 1 + max_raw_cep, grad_common)

    fig_w = 33/2.54
    fig_h = 2.0 + 1.65 * n
    fig = plt.figure(figsize=(fig_w, fig_h), dpi=100)

    header_title_r = 0.38
    header_sub_r   = 0.45
    footer_r       = 1.2
    title_r        = 0.20

    legend_rs = []
    for (label, data, grad) in selected:
        m = len(data.get(f"labels{label}", []))
        if m == 0: legend_rs.append(0.12)
        elif m <= 3: legend_rs.append(0.18)
        elif m <= 7: legend_rs.append(0.24)
        elif m <= 10: legend_rs.append(0.34)
        else: legend_rs.append(0.45)

    spacer_between = 0.35
    spacer_before_footer = 0.7
    FOND_ALPHA = AlphaFond_widget.value

    height_ratios = [header_title_r, header_sub_r]
    for i, lr in enumerate(legend_rs):
        height_ratios.extend([title_r, lr, 1.0])
        height_ratios.append(spacer_before_footer if i == n-1 else spacer_between)
    height_ratios.append(footer_r)

    total_rows = 2 + 4*n + 1
    gs = fig.add_gridspec(nrows=total_rows, ncols=1, height_ratios=height_ratios, hspace=0.03)

    ax_title = fig.add_subplot(gs[0, 0]); ax_title.axis('off')
    ax_sub   = fig.add_subplot(gs[1, 0]); ax_sub.axis('off')
    ax_title.text(0.0, 0.05, titre, ha='left', va='bottom', fontweight='bold', fontsize=16)
    ax_sub.text(0.0, 0.95, sousTitre, ha='left', va='top', fontsize=11) #ICICICICICICICICICIICICICICICICICICICICICIICIC

    for idx, (label, data, grad) in enumerate(selected):
        base_row = 2 + 4*idx
        ax_t = fig.add_subplot(gs[base_row, 0]); ax_t.axis('off')
        ax_l = fig.add_subplot(gs[base_row+1, 0]); ax_l.axis('off')
        ax   = fig.add_subplot(gs[base_row+2, 0])
        ax_s = fig.add_subplot(gs[base_row+3, 0]); ax_s.axis('off')
        ax_t.text(0.0, 0.5, label, ha='left', va='center', fontweight='bold', fontsize=12)
        ax.barh(
            1,
            data[f"valuesFond{label}"],
            left=data[f"gaucheFond{label}"],
            height=heightFond,
            color=data[f"extended_colors{label}"],
            alpha=FOND_ALPHA,
            edgecolor="none"
        )

        left_pos = 0
        for i, val in enumerate(data[f"values{label}"]):
            ax.barh(
                1,
                val,
                color=data[f"colors{label}"][i],
                left=left_pos,
                height=HeightBar,
                label=data[f"labels{label}"][i],
                zorder=3,
                edgecolor=SEGMENT_EDGE_COLOR,
                linewidth=SEGMENT_EDGE_WIDTH
            )
            left_pos += val

        ax.axvline(data[f"{label}max"], color="black", linewidth=2)
        ax.text(1.01 * data[f"{label}max"], 1,
                f"{label} max{data[f'{label}max']}",
                fontsize=7.5, va="center")

        if label in ("CEP", "CEPnr"):
            ax.set_xlim(0, xlim_cep)
            ax.set_xticks(xticks_cep)
        else:
            # clé valeur totale
            key_tot = {"Bbio": "bbio", "IcE": "IcE", "IcC": "IcC", "DH": "DH"}.get(label, label)
            valeur = data.get(key_tot, 0)
            seuil = data.get(f"{label}max", 0)
            ax.set_xlim(0, 1.05 * max(seuil, valeur))
            ax.set_xticks(np.arange(0, 1 + max(seuil, valeur), grad))

        ax.set_yticks([])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(True)
        ax.tick_params(axis='x', labelsize=8, pad=1)

        handles, labels_txt = ax.get_legend_handles_labels()

        if labels_txt:
            m = len(labels_txt)
        
            # ✅ règle simple et stable
            MAX_ONE_LINE = 7   # ajuste si tu veux 6 ou 8, mais UNE SEULE valeur
        
            if m <= MAX_ONE_LINE:
                ncol = m               # 1 seule ligne
            else:
                ncol = int(np.ceil(m / 2))  # 2 lignes équilibrées
        
            ax_l.legend(
                handles,
                labels_txt,
                loc="center left",
                ncol=ncol,
                frameon=False,
                fontsize=8.5,
                columnspacing=0.7,
                handletextpad=0.3,
                handlelength=0.8,
                borderaxespad=0.0
            )

    ax_footer = fig.add_subplot(gs[-1, 0]); ax_footer.axis('off')
    # note en haut de la zone footer
    ax_footer.text(0.0, 0.98, note, ha='left', va='top', fontsize=8,linespacing=1.15, wrap=True)
    # source en bas de la zone footer
    ax_footer.text(0.0, 0.02, infos, ha='left', va='bottom', fontsize=8, style='italic')

    fig.savefig(f"{titre}_Graph.png", dpi=250)
    plt.show()
    display(FileLink(f"{titre}_Graph.png"))


In [257]:
# === UI (Voila) ===

bouton_graph = widgets.Button(description="Mettre à jour le graphique", button_style='success', layout=widgets.Layout(width='240px'))
output_graph = widgets.Output()

def on_button_click(b):
    with output_graph:
        output_graph.clear_output(wait=True)

        nbRange = nbRange_widget.value
        HeightBar = HeightBar_widget.value
        heightFond = heightFond_widget.value
        titre = titre_widget.value
        sousTitre = sousTitre_widget.value
        infos = infos_widget.value
        grads_raw = [int(x.strip()) for x in graduationx_widget.value.split(",")]
        # compat: ancien format 5 valeurs (bbio, cep, cepnr, ic énergie, dh)
        if len(grads_raw) == 5:
            graduationx = [grads_raw[0], grads_raw[1], grads_raw[4], grads_raw[2], grads_raw[3], grads_raw[3]]
        else:
            graduationx = grads_raw

        d_bbio = calcul_donnees_bbio(nbRange)
        d_CEP = calcul_donnees_CEP(nbRange)
        d_CEPnr = calcul_donnees_CEPnr(nbRange)
        d_IcE = calcul_donnees_IcE(nbRange)
        d_IcC = calcul_donnees_IcC(nbRange)
        d_DH = calcul_donnees_DH(nbRange)

        afficher_graphique(d_bbio, d_CEP, d_CEPnr, d_IcE, d_IcC, d_DH,
                          nbRange, HeightBar, heightFond, graduationx,
                          titre, sousTitre, infos)

        selected_keys = []
        if show_bbio_widget.value: selected_keys.append("Bbio")
        if show_dh_widget.value: selected_keys.append("DH")
        if show_cep_widget.value: selected_keys.append("CEP")
        if show_cepnr_widget.value: selected_keys.append("CEPnr")
        if show_ice_widget.value: selected_keys.append("IcE")
        if show_icc_widget.value: selected_keys.append("IcC")
        recap_file = generer_recap_re2020(d_bbio, d_CEP, d_DH, d_CEPnr, d_IcE, d_IcC, titre, selected_keys=selected_keys)

        if recap_file is not None:
            display(Image(filename=recap_file))
            display(FileLink(recap_file))

bouton_graph.on_click(on_button_click)

# Style global
style_desc = {'description_width': '200px'}
large_layout = widgets.Layout(width='420px')

all_widgets = [
    BbioChaud_widget, BbioFroid_widget, BbioEcl_widget, Bbiomax_widget,
    CEPch_widget, CEPfr_widget, CEPecs_widget, CEPecl_widget, CEPauxV_widget,
    CEPauxD_widget, CEPdep_widget, CEPmax_widget,
    CEPnrch_widget, CEPnrfr_widget, CEPnrecs_widget, CEPnrecl_widget,
    CEPnrauxV_widget, CEPnrauxD_widget, CEPnrdep_widget, CEPnrmax_widget,
    cepnr_equals_cep_widget,
    IcEelec_widget, IcEbois_widget, IcErcu_widget, IcEfioul_widget,
    IcEgaz_widget, IcEmax_widget,
    IcC_lot01_widget, IcC_lot02_widget, IcC_lot03_widget, IcC_lot04_widget, IcC_lot05_widget,
    IcC_lot06_widget, IcC_lot07_widget, IcC_lot08_widget, IcC_lot09_widget, IcC_lot10_widget,
    IcC_lot11_widget, IcC_lot12_widget, IcC_lot13_widget, IcCmax_widget, show_icc_widget,
    DH_widget, DHmax_widget,
    titre_widget, sousTitre_widget, infos_widget,
    nbRange_widget, HeightBar_widget, heightFond_widget, graduationx_widget, AlphaFond_widget
]

for w in all_widgets:
    w.layout = large_layout
    w.style = style_desc

section_info = widgets.VBox([
    widgets.HTML("<h2>Informations générales</h2>"),
    titre_widget, sousTitre_widget, infos_widget
])

section_bbio = widgets.VBox([
    widgets.HTML("<h2>Valeurs Bbio</h2>"),
    BbioChaud_widget, BbioFroid_widget, BbioEcl_widget, Bbiomax_widget, show_bbio_widget
])

section_cep_group = widgets.VBox([
    widgets.HTML("<h2>Valeurs CEP</h2>"),
    CEPch_widget, CEPfr_widget, CEPecs_widget, CEPecl_widget,
    CEPauxV_widget, CEPauxD_widget, CEPdep_widget, CEPmax_widget, show_cep_widget,
    widgets.HTML("<h2>Valeurs CEPnr</h2>"),
    CEPnrch_widget, CEPnrfr_widget, CEPnrecs_widget, CEPnrecl_widget,
    CEPnrauxV_widget, CEPnrauxD_widget, CEPnrdep_widget, CEPnrmax_widget,
    show_cepnr_widget,
    cepnr_equals_cep_widget
])

section_ice = widgets.VBox([
    widgets.HTML("<h2>Valeurs IcE</h2>"),
    IcEelec_widget, IcEbois_widget, IcErcu_widget, IcEfioul_widget,
    IcEgaz_widget, IcEmax_widget,show_ice_widget
])



section_icc = widgets.VBox([
    widgets.HTML("<h2>Valeurs Ic construction</h2>"),
    IcC_lot01_widget, IcC_lot02_widget, IcC_lot03_widget, IcC_lot04_widget,
    IcC_lot05_widget, IcC_lot06_widget, IcC_lot07_widget, IcC_lot08_widget,
    IcC_lot09_widget, IcC_lot10_widget, IcC_lot11_widget, IcC_lot12_widget,
    IcC_lot13_widget, IcCmax_widget, show_icc_widget
])

section_dh = widgets.VBox([
    widgets.HTML("<h2>Valeurs DH</h2>"),
    DH_widget, DHmax_widget, show_dh_widget
])

section_graph_params = widgets.VBox([
    widgets.HTML("<h2>Paramètres graphiques</h2>"),
    nbRange_widget, HeightBar_widget, heightFond_widget, graduationx_widget, AlphaFond_widget
])

accordion = widgets.Accordion(children=[
    section_info, section_bbio, section_cep_group, section_dh, section_ice, section_icc, section_graph_params
])
accordion.set_title(0, 'Informations générales')
accordion.set_title(1, 'Valeurs Bbio')
accordion.set_title(2, 'Valeurs CEP & CEPnr')
accordion.set_title(3, 'Valeurs DH')
accordion.set_title(4, 'Valeurs IcE')
accordion.set_title(5, 'Valeurs Ic construction')
accordion.set_title(6, 'Paramètres graphiques')

ui = widgets.VBox([accordion, bouton_graph, output_graph])
display(ui)
